In [1]:
import torch
import torch.nn
import numpy as npe
import seaborn as sns
from Losses import distance_matrix_vector
from ReprojectionStuff import get_GT_correspondence_indexes
from ContPatch import MakeContourPatches
from  torch.autograd import Variable as V
from ContPatch import denormalizeContour,normalizeContour
from ContPatch import DrawContourBoundaries
#%matplotlib inline
import seaborn as sns
#plt.imshow(255 - img1.cpu().data.numpy()[0,0,:,:])
from LAF import visualize_LAFs
def loss_HardNetDetach(anchor, positive, margin = 1.0):
    assert anchor.size() == positive.size(), "Input sizes between positive and negative must be equal."
    assert anchor.dim() == 2, "Inputd must be a 2D matrix."
    eps = 1e-8
    dist_matrix = distance_matrix_vector(anchor, positive) + eps
    pos1 = torch.diag(dist_matrix)
    eye = torch.autograd.Variable(torch.eye(dist_matrix.size(1)))
    if anchor.is_cuda:
        eye=eye.cuda()
    dist_matrix_detach = distance_matrix_vector(anchor, positive.detach()) + eps
    # steps to filter out same patches that occur in distance matrix as negatives
    dist_without_min_on_diag = dist_matrix_detach + eye*10
    mask = (dist_without_min_on_diag.ge(0.008).float()-1)*-1
    mask = mask.type_as(dist_without_min_on_diag)*10
    dist_without_min_on_diag = dist_without_min_on_diag + mask
    min_neg = torch.min(dist_without_min_on_diag,1)[0]
    min_neg = min_neg
    loss = torch.clamp(margin + pos1 - min_neg, min=0.0)
    loss = torch.mean(loss)
    return loss


In [2]:
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import os
from PIL import Image
import math
from copy import deepcopy
import gc
from subprocess import call
from ContPatch import GetContoursPatchGrid,GetKLongestFromM, makeContoursFromMNumpy
class L2Norm(nn.Module):
    def __init__(self):
        super(L2Norm,self).__init__()
        self.eps = 1e-8
    def forward(self, x):
        norm = torch.sqrt(torch.sum(x * x, dim = 1) + self.eps)
        x = x / norm.unsqueeze(1).expand_as(x)
        return x
def get_input_param_optimizer(x, lr):
    input_param = nn.Parameter(x.data)
    optimizer = optim.Adam([input_param], lr=lr)
    return input_param, optimizer

def DrawContourBoundaries_on_fig(ax, conts, c = ('g', 'b')):
    ph = 41
    grid = GetContoursPatchGrid(conts, ph).transpose(1,2)
    lines = {}
    idxs1 = np.unique(conts[:,2].cpu().numpy())
    for i in idxs1:
        i = int(i)
        idxs = torch.nonzero(conts[:,2] == int(i)).long()
        if len(idxs) < 1:
            continue
        boundary_x = np.concatenate([grid[0,0, idxs, 0].data.cpu().numpy(), 
                                     grid[0,ph-1,idxs,0].data.cpu().numpy()[::-1]])
        boundary_y = np.concatenate([grid[0,0, idxs, 1].data.cpu().numpy(),
                                     grid[0,ph-1,idxs,1].data.cpu().numpy()[::-1]])
        boundary_x = np.concatenate([boundary_x,boundary_x[0:1]])
        boundary_y = np.concatenate([boundary_y,boundary_y[0:1]])
        lines[str(i)] = []
        lines[str(i)].append(ax.plot(boundary_x, boundary_y, c[0]))
        lines[str(i)].append(ax.plot(grid[0,ph/2, idxs, 0].data.cpu().numpy(),
                                     grid[0,ph/2, idxs,1].data.cpu().numpy(), c[1]))
    return lines, ax

from HardNet import HardNet
class ContDiscrOptimDetach():
    def __init__(self, descriptors_list, names_list, cuda = False, lr = 1e-0):
        assert len(descriptors_list) == len(names_list)
        self.desc_list = descriptors_list
        self.names_list = names_list
        self.colors = ['r', 'g', 'b', 'y', 'm', 'k', 'c']
        self.cuda = cuda
        self.out_conts1 = {}
        self.out_conts2 = {}
        self.loss_func = loss_HardNetDetach
        self.out_loss_mean = {}
        self.rad_diff = {}
        self.nc_diff = {}
        self.lr = lr;
        for dn in names_list:
            self.out_conts1[dn] = []
            self.out_conts2[dn] = []      
            self.out_loss_mean[dn] =[]
            self.rad_diff[dn] = []
            self.nc_diff[dn] =[]
        return
    def optimize(self, c1, c2, img1, img2, n_iters = 10):
        if self.cuda:
            img1,img2 = img1.cuda(),img2.cuda()
        w1,h1 = img1.size(3),img1.size(2)
        w2,h2 = img2.size(3),img2.size(2)
        for d_idx in range(len(self.names_list)):
            D = self.desc_list[d_idx]
            try:
                if self.cuda:
                    D = D.cuda()
            except:
                pass
            N = self.names_list[d_idx]
            print N
            cc1 = deepcopy(Variable(deepcopy(c1)))
            cc2 = deepcopy(Variable(deepcopy(c2)))
            if self.cuda:
                cc1 = cc1.cuda()
                cc2 = cc2.cuda()
            cont_iii = np.unique(cc1[:,2].data.cpu().numpy())
            num_cont = len(cont_iii)
            
            c1o, opt1 = get_input_param_optimizer(cc1[0:num_cont,5:], self.lr)
            c2o, opt2 = get_input_param_optimizer(cc2[0:num_cont,5:], self.lr)
            use_cont1 = V(torch.zeros(cc1.size()).float())
            use_cont2 = V(torch.zeros(cc1.size()).float())
            
            if self.cuda:
                use_cont1 = use_cont1.cuda()
                use_cont2 = use_cont2.cuda()
            for ci in range(num_cont):
                iiii = int(cont_iii[ci])
                ooooo = cc1[:,2] == iiii;
                use_cont1[ooooo,:] = torch.cat([cc1[ooooo,:5],c1o[ci,:].repeat(ooooo.long().sum().data[0], 1)], dim = 1)
            for ci in range(num_cont):
                iiii = int(cont_iii[ci])
                ooooo = cc2[:,2] == iiii;
                use_cont2[ooooo,:] = torch.cat([cc2[ooooo,:5],c2o[ci,:].repeat(ooooo.long().sum().data[0], 1)], dim = 1)
            self.out_conts1[N].append(deepcopy(use_cont1.data))
            self.out_conts2[N].append(deepcopy(use_cont2.data))
            for it in range(n_iters):
                use_cont1 = V(torch.zeros(cc1.size()).float())
                use_cont2 = V(torch.zeros(cc1.size()).float())
                if self.cuda:
                    use_cont1 = use_cont1.cuda()
                    use_cont2 = use_cont2.cuda()
                for ci in range(num_cont):
                    iiii = int(cont_iii[ci])
                    ooooo = cc1[:,2] == iiii;
                    use_cont1[ooooo,:] = torch.cat([cc1[ooooo,:5],c1o[ci,:].repeat(ooooo.long().sum().data[0], 1)], dim = 1)
                for ci in range(num_cont):
                    iiii = int(cont_iii[ci])
                    ooooo = cc2[:,2] == iiii;
                    use_cont2[ooooo,:] = torch.cat([cc2[ooooo,:5],c2o[ci,:].repeat(ooooo.long().sum().data[0], 1)], dim = 1)
                p1 = MakeContourPatches(img1, use_cont1,  32)
                desc1 = V(torch.zeros(num_cont,128))
                desc2 = V(torch.zeros(num_cont,128))
                for ci in range(num_cont):
                    iiii = int(cont_iii[ci])
                    ooooo = use_cont1[:,2] == iiii;
                    desc1[ci,:] = D(p1[0,0,ooooo,:].unsqueeze(0).unsqueeze(0))
                p2 = MakeContourPatches(img2, use_cont2,  32)
                for ci in range(num_cont):
                    iiii = int(cont_iii[ci])
                    ooooo = use_cont2[:,2] == iiii;
                    desc2[ci,:] = D(p2[0,0,ooooo,:].unsqueeze(0).unsqueeze(0))
                loss = self.loss_func(desc1,desc2)
                #torch.sqrt(((desc1.view(-1,1)-desc2.view(-1,1))**2).sum(0, keepdim=True) + 1e-12).mean()
                if it % 10 == 0:
                    print loss.data.cpu().numpy()[0]
                opt1.zero_grad()
                opt2.zero_grad()
                loss.backward() 
                opt1.step()
                opt2.step()
                self.out_conts1[N].append(deepcopy(use_cont1.data))
                self.out_conts2[N].append(deepcopy(use_cont2.data))
                self.out_loss_mean[N].append(deepcopy(loss.data.mean()))
                self.rad_diff[N].append(deepcopy(torch.sqrt((c1o-c2o)**2 + 1e-10).data.mean()))
            del cc1,cc2,opt1,opt2
            gc.collect()
            torch.cuda.empty_cache()
        self.img1 = img1.data.cpu().numpy()
        self.img2 = img2.data.cpu().numpy()
        return
    def save_data(self, fname):
        data_dict = {'LAFs1':self.out_lafs1, 'LAFs2':self.out_lafs2, 'loss_mean': self.out_loss_mean,
                    'rad_diff': self.rad_diff, 'shape_diff': self.shape_diff}
        with open(fname, 'wb') as ff:
            pickle.dump(data_dict, ff)
        return
    def load_data(self, fname):
        with open(fname, 'rb') as ff:
            data_dict = pickle.load(ff)
        self.out_lafs1 = data_dict['LAFs1']
        self.out_lafs2 = data_dict['LAFs2']
        self.out_loss_mean = data_dict['loss_mean']
        self.rad_diff = data_dict['rad_diff']
        self.shape_diff = data_dict['shape_diff']
        return    
    def savemp4_per_desc(self, fname):
        img = self.img1
        for d_idx in range(len(self.names_list)):
            fig, ax = plt.subplots()
            fig.set_tight_layout(True)
            ax.imshow(255 - img.squeeze())
            lines_dict = {}
            N = self.names_list[d_idx];
            C = self.colors[d_idx];
            lafs_dict = {"1": self.out_conts1[N], "2": self.out_conts2[N]}
            num_frames = len(lafs_dict["1"])
            lines_dict['1'], ax =  DrawContourBoundaries_on_fig(ax, lafs_dict["1"][0], ('g','b'))
            lines_dict['2'], ax =  DrawContourBoundaries_on_fig(ax, lafs_dict["2"][0], ('m','c'))
            #ax.legend(['img1', 'img2'])
            def visualize_LAFs_on_fig_mod(laf_dict, line_dictss, dname, i):
                work_conts = V(laf_dict[dname][i])
                line_dict = line_dictss[dname]
                ph = 41
                grid = GetContoursPatchGrid(work_conts, ph).transpose(1,2)
                for jj in np.unique(work_conts[:,2].data.cpu().numpy()):
                    jj = int(jj)
                    idxs = torch.nonzero(work_conts[:,2] == int(jj)).long()
                    if len(idxs) < 1:
                        continue
                    boundary_x = np.concatenate([grid[0,0, idxs, 0].data.cpu().numpy(), 
                                                 grid[0,ph-1,idxs,0].data.cpu().numpy()[::-1]])
                    boundary_y = np.concatenate([grid[0,0, idxs, 1].data.cpu().numpy(),
                                                 grid[0,ph-1,idxs,1].data.cpu().numpy()[::-1]])
                    boundary_x = np.concatenate([boundary_x,boundary_x[0:1]])
                    boundary_y = np.concatenate([boundary_y,boundary_y[0:1]])
                    line_dict[str(jj)][0][0].set_data(boundary_x, boundary_y)
                    line_dict[str(jj)][1][0].set_data(grid[0,ph/2, idxs, 0].data.cpu().numpy(),
                                                 grid[0,ph/2, idxs,1].data.cpu().numpy())
                return line_dictss[dname],ax
            def update_LAF(i):
                lines_dict["1"], ax =  visualize_LAFs_on_fig_mod(lafs_dict, lines_dict, "1", i)
                lines_dict["2"], ax =  visualize_LAFs_on_fig_mod(lafs_dict, lines_dict, "2", i)
                return ax 
            anim = FuncAnimation(fig, update_LAF, frames=np.arange(0, num_frames), interval=75)
            Writer = animation.writers['ffmpeg']
            writer = Writer(fps=24, metadata=dict(artist='Me'), bitrate=900)
            anim.save('output_discrim_align_detach_conts/' + N + "_" +  fname, dpi=72, writer=writer)#'imagemagick')
        return


In [3]:
input_img_fname1 = '/home/old-ufo/Dropbox/HP/i_indiana/1.ppm'
input_img_fname2 = '/home/old-ufo/Dropbox/HP/i_indiana/6.ppm'

def load_grayscale_var(fname):
    img = Image.open(fname).convert('RGB')
    img = np.mean(np.array(img), axis = 2)
    var_image = torch.autograd.Variable(torch.from_numpy(img.astype(np.float32)), volatile = True)
    var_image_reshape = var_image.view(1, 1, var_image.size(0),var_image.size(1))
    if False:
        var_image_reshape = var_image_reshape.cuda()
    return var_image_reshape

import math

d1 =  lambda x: L2Norm()(x.view(x.size(0), -1))


d2 = HardNet()
model_weights = 'HardNet++.pth'
hncheckpoint = torch.load(model_weights)
d2.load_state_dict(hncheckpoint['state_dict'])
d2.eval()
    
desc_list = [d2]#,d2,d3,d4]
desc_names = ['HardNetMaxPool']#, 'HardNet', 'SIFT', 'TFeat']


In [5]:
def contours_from_matlab(img_fname):
    c_str =  "detect_and_save_contour_single('"+ img_fname  +"', '" +os.getcwd() + "/temp.mat" +"'); exit()" 
    print c_str
    cwd = os.getcwd() 
    os.chdir('/home/old-ufo/dev/line_Jimmy/dlines/matlab/')
    call(["/home/old-ufo/apps/MATLAB/bin/matlab", "-nodisplay",
          "-nojvm", "-r", c_str]);
    from scipy import io
    out = io.loadmat(cwd + "/temp.mat")['M']
    os.chdir(cwd)
    return out
import cPickle as pickle
dn = '/home/old-ufo/Dropbox/HP/'
for dir1 in sorted(os.listdir(dn)):
    if 'i_' not in dir1:
        continue
    print dir1
    if os.path.isfile(dir1 + '.pickle'):
        continue
    if os.path.isfile('output_discrim_align_detach_conts_un/HardNetMaxPool_'+dir1 + '_single.mp4'):
        continue
    img1 = load_grayscale_var(os.path.join(dn,dir1) + '/1.ppm') 
    img2 = load_grayscale_var(os.path.join(dn,dir1) + '/2.ppm')
    yy1 = contours_from_matlab(os.path.join(dn,dir1) + '/1.ppm');
    yy1 = denormalizeContour(yy1, img1.size(3), img1.size(2))
    
    yy1 = GetKLongestFromM(yy1, 30)
    yy2 = deepcopy(yy1)
    c1 = makeContoursFromMNumpy(yy1, 30.0)
    c2 = makeContoursFromMNumpy(yy2, 30.0)
    LOP = ContDiscrOptimDetach(desc_list, desc_names, cuda = True, lr = 2.0)
    LOP.optimize(c1.data, c2.data, img1, img2, 100)
    LOP.img1 = img1.data.cpu().numpy()
    LOP.savemp4_per_desc(dir1 + '_single.mp4')  
    #LOP.save_data(dir1 + '.pickle');
    #print 'opt done'
    #break
    #LOP.savemp4_per_desc(dir1 + '.mp4')  

i_ajuntament
detect_and_save_contour_single('/home/old-ufo/Dropbox/HP/i_ajuntament/1.ppm', '/home/old-ufo/dev/scalenet/contour/temp.mat'); exit()


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


HardNetMaxPool
0.820673
0.786134
0.768818
0.755725
0.750447
0.748164
0.7469
0.747541
0.747412
0.746795
i_autannes
detect_and_save_contour_single('/home/old-ufo/Dropbox/HP/i_autannes/1.ppm', '/home/old-ufo/dev/scalenet/contour/temp.mat'); exit()
HardNetMaxPool
0.744791
0.746687
0.731726
0.723662
0.71829
0.711069
0.70986
0.710702
0.707785
0.709545


KeyboardInterrupt: 

In [19]:
LOP.img1 = img1.data.cpu().numpy()
LOP.savemp4_per_desc(dir1 + '_single_mp22.mp4')  

In [5]:
print '1'

1


i_ajuntament
detect_and_save_contour_single('/home/old-ufo/Dropbox/HP/i_ajuntament/1.ppm', '/home/old-ufo/dev/scalenet/contour/temp.mat'); exit()


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Pixels
